# 1.导入库

In [1]:
from transformers import Trainer, TrainingArguments,AutoTokenizer, AutoModelForSequenceClassification

from datasets import load_dataset


# 2.加载数据集 和 划分数据

In [3]:
from datasets import load_dataset

from datasets import load_dataset

dataset = load_dataset("dirtycomputer/ChnSentiCorp_htl_all",split="train")


In [19]:
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7766
})

In [4]:
datasets = dataset.train_test_split(test_size=0.2)
datasets


DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6212
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 1554
    })
})

# Filter None data from the dataset

In [5]:
datasets = datasets.filter(lambda example: example['review'] is not None)

Filter: 100%|██████████| 1554/1554 [00:00<00:00, 68602.76 examples/s]


# create dataloader

In [13]:
import torch 
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def tokenize_function(examples):
    if examples['review']:
        print(examples['review'])
        tokeninzer_examples = tokenizer(examples['review'],max_length=256,truncation=True)  #  truncation=True 表示截断文本
        tokeninzer_examples['label'] = examples['label']
        return tokeninzer_examples

tokenizer_dataset = datasets.map(tokenize_function, batched=True,remove_columns=datasets['train'].column_names)
tokenizer_dataset

Map: 100%|██████████| 1554/1554 [00:00<00:00, 9875.71 examples/s] 

['酒店整体还不错,豪华套房整体的感觉是客厅稍大了些,卧房小了些.其余都还可以.', '我住过南京所有的五星酒店,选来选去还是这家(曾经的香格里拉).无论是硬件设施还是服务,都很无可挑剔...如果说缺点,就是房价很贵了,呵呵,但愿携程能谈下更优惠的房价啊.', '还不错得酒店，我去过夏威夷，那里不如这里，这里得感觉才像4星的，就是前台的女孩不热情，很冷淡的接待你。我住的350的双人房间，床很大，房间也很宽敞，每天又2瓶免费的矿泉水，有保险箱，有小冰箱，电视是25寸的，能免费上网，电视也能上网，有星空电影，凤凰卫视一些频道，就是没有BBC，CNN什么的。早餐还算一般，比较丰富，就是没有我爱吃的东西。对了电视上网还有一个无线的小键盘，比较有意思。这里还有很多房型，比如无烟层，女性层，A型标准间，普通标准间等等，不知道携程为什么只提供了这么少的房型。我本来要订10天的房子，可是只给订5天，入住后再续住，比较麻烦，而且我订之后发现金鼎和夏威夷在携程上都显示满了，可是我在酒店前台问，还有很多空房可以住，不知道这是怎么会事。金鼎的餐厅饭菜还可以，夏威夷我也去吃了，和我们一起的老美说，夏威夷太难吃了，还是金鼎好些。这里交通很方便，距离步行街很近，但不吵，夏威夷离步行街太近了，比较吵，也比较闹一些。还有，我对携程订房有些意见，如果我入住时间比较长，酒店那方就不让订了，我只能先入住，然后再续住～，不太合理。牡丹江还有一些酒店，比如远东，就在夏威夷对面，携程上不能订，金鼎对面和斜对面要开2个比较大的酒店，看装修应该是4星了，不知道怎么样。点评完毕，谢谢收看。不到支持请见谅。', '非常满意酒店的服务，做得很细心，比其它酒店好多了，以后还会再住在此酒店，谢谢携程的工作人员帮忙让我们过得很愉快，在此要特别感谢酒店在3月28号这天对我们的最贴心服务，因这天是我们的大喜日子，早上从外面拍完照片回来我的拖尾婚纱太长了搞的很dirty，打电话叫他们帮加急洗，后来帮我洗得很快很干净地送回来还free，非常感动，再次谢谢他们的工作人员，该酒店各方面都很好，唯一的遗憾就是大堂的酒吧，东西很少，想要喝的东西都没有，坐那里都没有意思，希望下次他们能改进，不要总是写满在酒水单上让人看什么都有，事实上都没有供应，', '旋宫酒店的性价比还是比较高的（我这次住的是235元），主要是位置实在很好，离佛山的主要景点祖

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6211
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1554
    })
})

# 创建模型

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    'hfl/rbt3'
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.config

BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

In [9]:
import evaluate


In [10]:
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [16]:
import numpy as np
def eval_metric(eval_predict):
    predictions,labels = eval_predict
    predictions = np.argmax(predictions, axis=1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1.metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

C:\Users\zata\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level

# create a trainer 

In [19]:
from transformers import DataCollatorWithPadding
traniner = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenizer_dataset['train'],
    eval_dataset=tokenizer_dataset['test'],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric
)
    

# model training

In [20]:
traniner.train()

  2%|▏         | 6/294 [01:05<52:47, 11.00s/it]

: 